In [2]:
import sys
sys.path.append('../..')
import torch
import numpy as np
from PIL import Image
import glob
from torch import cuda
import os
from os.path import join
import shutil
from DLBio.pytorch_helpers import get_device
import ds_ear_siamese as des
import transforms_data as td

In [3]:
FOLDER_COMPARE = './auth_dataset/'
PATH_TO_ALL = '../dataset/'
DEVICE = get_device()
# net = torch.load('./models/model.pt', DEVICE)
#model = SiameseNetwork()

In [4]:
# method for displaying files with index
def print_list(list_):
    fmt = '{:<8}{:<20}'
    print(fmt.format('Index', 'Name'))
    for i, name in enumerate(list_):
        print(fmt.format(i, name))

In [5]:
# takes the name input and returns full path to user
# if no name is selected, a list to choose from is presented
def choose_user(name='unknown'):
    users = os.listdir(PATH_TO_ALL)
    user_name = name
    # if statement starts, when no name was given
    if user_name == 'unknown':
        print_list(users)
        # Handle the user's input for user name
        while True:
            idx = input('Choose your user name by index: ')
            try:
                idx = int(idx)
                assert idx < len(users) and idx >= 0
                break
            except (ValueError, AssertionError):
                print('The input was a string or not in the index range.')
        user_name = users[idx]
    
    assert user_name in users, 'The name was not found in the dataset.'
    
    return os.path.join(PATH_TO_ALL, user_name)

In [ ]:
# Image processing
# input is the filename and the transforamtion function
def image_pipeline(input_, preprocess):
    input_ = Image.open(input_)
    input_ = preprocess(input_)
    input_ = input_.reshape(-1, RESIZE_Y, RESIZE_X, 1)
    input_ = input_.permute(3, 0, 1, 2)

    if cuda.is_available():
        return input_.type('torch.cuda.FloatTensor')
    else:
        return input_.type('torch.FloatTensor')

In [ ]:
# Setting up the dataset
def get_triplets(dataset_path, user_name, verif_dataset, transform_mode ):
    NUM_CLASSES = len(os.listdir( dataset_path ))
    user_imgs = os.listdir(join(dataset_path, user_name))
    
    preprocess = td.transforms_siamese_verification( (100, 100) )

    # NUM_EXAMPLES is the number of (A,P,N) triplets chosen for the same class (N belongs to a different class of course)
    NUM_EXAMPLES = 5

    # Triplets list will contain anchor(A), positive(P) and negative(N) triplets.
    triplets = []
    # assert len(os.listdir(verif_dataset)) < len(os.listdir(user_path)), 'There are more verification images than po'
    # creating anchor, positive, negative triplets (amount equals taken images)
    for img_dir in os.listdir( verif_dataset ):
        anchor_dir = join(verif_dataset, img_dir)
        anchor = image_pipeline(anchor, preprocess)

        # choose random image from the alleged user and pass the preprocess strategy
        positive = image_pipeline( join(dataset_path, user_imgs[np.random.randint(0, len(user_imgs))]), preprocess )
        
        # find a class different from anchor. if same as anchor try again
        neg_class = os.listdir(dataset_path)[np.random.randint(NUM_CLASSES)]
        while neg_class == anchor_dir: 
            neg_class = os.listdir(dataset_path)[np.random.randint(NUM_CLASSES)]
        # list of image file names
        neg_dir_imgs = os.listdir(join(dataset_path, neg_class))
        # choose random image from the negative and pass the preprocess strategy
        negative = image_pipeline( join(dataset_path, neg_dir, neg_dir_imgs[np.random.randint(0, len(neg_dir_imgs))]), preprocess )

        # append triplet
        triplets.append([anchor, positive, negative])

        return triplets

In [ ]:
RESIZE_Y = 100
RESIZE_X = 100
image_array = []
files = glob.glob ('../temp_verification')
files.sort()
# declare function of transformation
preprocess = td.transforms_valid_and_test((RESIZE_Y, RESIZE_X))

for f in files:
    image = Image.open(f)
    image_transformed = preprocess(image)
    image_transformed = image_transformed.reshape(-1, RESIZE_Y, RESIZE_X, 1)
    image_transformed = image_transformed.permute(3, 0, 1, 2)
    if cuda.is_available():
        image_array.append(image_transformed.type('torch.cuda.FloatTensor'))
    else:
        image_array.append(image_transformed.type('torch.FloatTensor'))


In [ ]:
from torch.utils.data import DataLoader, SubsetRandomSampler, Dataset
import torchvision
import transforms_data as td
from torchvision import datasets

new_img_dset = torchvision.datasets.ImageFolder( root = '../temp_verification' )


In [ ]:
valid_dataset = SiameseNetworkDataset(imageFolderDataset=FOLDER_TEST_COMPARE,
                                        transform=transforms.Compose([transforms.Resize((100,100)),
                                                                      transforms.ToTensor()
                                                                      ])
                                       ,should_invert=False)

test_dataloader = DataLoader(siamese_dataset,num_workers=6,batch_size=1,shuffle=True)

# is the same as:
# test_dataloader = ds_ear_siamese.get_dataloader(indices=None, batch_size=1, num_workers=6, transform_mode='siamese',
#                                                  data_path=FOLDER_TEST_COMPARE, should_invert=False)

In [ ]:
folder_dataset_test = dset.ImageFolder(root=Config.testing_dir)
siamese_dataset = SiameseNetworkDataset(imageFolderDataset=folder_dataset_test,
                                        transform=transforms.Compose([transforms.Resize((100,100)),
                                                                      transforms.ToTensor()
                                                                      ])
                                       ,should_invert=False)

test_dataloader = DataLoader(siamese_dataset,num_workers=6,batch_size=1,shuffle=True)
dataiter = iter(test_dataloader)
x0,_,_ = next(dataiter)

for i in range(10):
    _,x1,label2 = next(dataiter)
    concatenated = torch.cat((x0,x1),0)
    
    output1,output2 = net(Variable(x0).cuda(),Variable(x1).cuda())
    euclidean_distance = F.pairwise_distance(output1, output2)
    imshow(torchvision.utils.make_grid(concatenated),'Dissimilarity: {:.2f}'.format(euclidean_distance.item()))